### Курсовая работа#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import os
import sys
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset, random_split
import torchvision.transforms as tt
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline
import time
from torch.autograd import Variable
import cv2
from facenet_pytorch import MTCNN

sys.path.insert(0, './yolo-hand-detection')

from yolo import YOLO

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
# Параметры для обучения
batch_size = 200
epochs = 10
max_lr = 0.008
grad_clip = 0.1
weight_decay = 1e-4

In [4]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [5]:
train_transforms = tt.Compose([tt.Grayscale(num_output_channels=1), # Картинки чернобелые
                         
                         tt.CenterCrop(250),
                         tt.Resize(100),
                               
                         # Настройки для расширения датасета
                         tt.RandomRotation(45),               # Случайные повороты на 30 градусов
                         tt.ToTensor()])                      # Приведение к тензору

#test_transforms = tt.Compose([tt.Grayscale(num_output_channels=1), tt.ToTensor()])

In [6]:
data_dir = '../data/leapGestRecog'
print(os.listdir(data_dir))

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']


In [7]:
digit_to_classname = {0:'palm', 1:'l', 2:'fist', 3:'fist_moved', 4:'thumb', 5:'index',
                      6:'ok', 7:'palm_moved', 8:'c', 9:'down'}
num_classes = len(digit_to_classname)

In [8]:
dataset = None

for folder in os.listdir(data_dir):
    if dataset is not None:
        data_to_add = ImageFolder(data_dir + f'/{folder}', train_transforms)
        dataset = ConcatDataset([dataset, data_to_add])
    else:
        dataset = ImageFolder(data_dir + f'/{folder}', train_transforms)

In [9]:
torch.manual_seed(2177)
num_of_elements = int(len(dataset))
train_part = 0.7 # часть датасета для обучения
train_dataset, test_dataset = random_split(dataset, [int(train_part * num_of_elements), int((1-train_part) * num_of_elements)])

In [10]:
train_dataset[0][0].shape

torch.Size([1, 100, 100])

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=3, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size, num_workers=3, pin_memory=True)

In [12]:
train_dataloader = DeviceDataLoader(train_dataloader, device)
test_dataloader = DeviceDataLoader(test_dataloader, device)

In [13]:
class Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.pool_2 = nn.MaxPool2d(2)
        self.pool_6 = nn.MaxPool2d(6)
        self.drop = torch.nn.Dropout(p=0.3)
        self.flat = nn.Flatten()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        
        self.fc1 = nn.Linear(3136, 512) #576 - 3136
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        
        x = F.relu(self.conv2(x))
        x = self.pool_2(x)
        x = self.drop(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool_6(x)
        x = self.drop(x)
        
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        
        return x

In [14]:
model = to_device(Net(1, num_classes), device)
model

Net(
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool_6): MaxPool2d(kernel_size=6, stride=6, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.3, inplace=False)
  (flat): Flatten()
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), max_lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_dataloader))

### Обучение

In [16]:
"""
epoch_losses = []

for epoch in range(epochs):
    
    time1 = time.time()
    running_loss = 0.0
    epoch_loss = []
    for batch_idx, (data, labels) in enumerate(train_dataloader):
        data, labels = Variable(data), Variable(labels)
        data = data.to(device)
        labels = labels.to(device)
        
        
        optimizer.zero_grad()
        
        outputs = model(data)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item()
        epoch_loss.append(loss.item())
        if (batch_idx+1) % 100 == 99:
            print(f'Train Epoch: {epoch+1}, Loss: {running_loss/10000}')
            time2 = time.time()
            print(f'Spend time for 10000 images: {time2 - time1} sec')
            time1 = time.time()
            running_loss = 0.0
    print(f'Epoch {epoch+1}, loss: ', np.mean(epoch_loss))
    epoch_losses.append(epoch_loss)
    """

"\nepoch_losses = []\n\nfor epoch in range(epochs):\n    \n    time1 = time.time()\n    running_loss = 0.0\n    epoch_loss = []\n    for batch_idx, (data, labels) in enumerate(train_dataloader):\n        data, labels = Variable(data), Variable(labels)\n        data = data.to(device)\n        labels = labels.to(device)\n        \n        \n        optimizer.zero_grad()\n        \n        outputs = model(data)\n        loss = F.cross_entropy(outputs, labels)\n        loss.backward()\n        optimizer.step()\n        scheduler.step()\n        \n        running_loss += loss.item()\n        epoch_loss.append(loss.item())\n        if (batch_idx+1) % 100 == 99:\n            print(f'Train Epoch: {epoch+1}, Loss: {running_loss/10000}')\n            time2 = time.time()\n            print(f'Spend time for 10000 images: {time2 - time1} sec')\n            time1 = time.time()\n            running_loss = 0.0\n    print(f'Epoch {epoch+1}, loss: ', np.mean(epoch_loss))\n    epoch_losses.append(epoch_l

In [17]:
# Сохранение модели
# torch.save(model.state_dict(), './Hand_gesture_recognition_model_100_state_10_epoch.pth')

In [18]:
net = Net(1, num_classes).to(device) 
net.load_state_dict(torch.load('./Hand_gesture_recognition_model_100_state_10_epoch.pth'))

<All keys matched successfully>

In [19]:
# Создаем объект для считывания потока с веб-камеры(обычно вебкамера идет под номером 0. иногда 1)
cap = cv2.VideoCapture(0)  

yolo = YOLO("yolo-hand-detection/models/cross-hands.cfg", "yolo-hand-detection/models/cross-hands.weights",
            ["hand"], confidence=0.5, threshold=0.3)

# Класс детектирования и обработки лица с веб-камеры 
class FaceDetector(object):

    def __init__(self, mtcnn):
        self.mtcnn = mtcnn
        self.hand_detector = yolo
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.emodel = Net(1, 10).to(self.device)
        self.emodel.load_state_dict(torch.load('./Hand_gesture_recognition_model_100_state_10_epoch.pth'))
        self.emodel.eval()

    # Функция рисования найденных параметров на кадре
    def _draw(self, frame, boxes, probs, landmarks):
        try:
            for box, prob, ld in zip(boxes, probs, landmarks):
                # Рисуем обрамляющий прямоугольник лица на кадре
                cv2.rectangle(frame,
                              (box[0], box[1]),
                              (box[2], box[3]),
                              (0, 0, 255),
                              thickness=2)
        except:
            pass
            #print('Something wrong im draw function!')

        return frame
    
 
    # Функция для вырезания рук и с кадра
    @staticmethod
    def crop_hand(frame, boxes, exp=0):
        hands = []
        for i, box in enumerate(boxes):
            hands.append(frame[int(box[1])-exp:int(box[3])+exp, 
                int(box[0])-exp:int(box[2])+exp])
        return hands
    
    # Словарь жестов
    @staticmethod
    def digit_to_classname(digit):
        digit_to_classname_dict = {0:'palm', 1:'l', 2:'fist', 3:'fist_moved', 4:'thumb', 5:'index',
                                  6:'ok', 7:'palm_moved', 8:'c', 9:'down'}
        return digit_to_classname_dict[digit]
    
    # Функция реакции на жест
    def gesture_action(self, digit, frame):
        if digit is None:
            return "", frame 
        
        gesture_to_show = self.digit_to_classname(digit)
        if digit in [3, 5, 7, 9]:  # Данные положения руки плохо определяются детектором, поэтому только надпись
            pass
        if digit == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if digit == 1:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        if digit == 2:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        if digit == 4:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if digit == 6:  
            pass  # Вернуть изначальный вариант
        if digit == 8:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        return gesture_to_show, frame
        
    # Функция в которой будет происходить процесс считывания и обработки каждого кадра
    def run(self):              
        gesture = ''
        gesture_to_show = ''
        gesture_idx = None
        guess_list = np.array([0] * 10)
        x, y = [0, 0]
        
        # Заходим в бесконечный цикл
        while True:
            # Считываем каждый новый кадр - frame
            # ret - логическая переменая. Смысл - считали ли мы кадр с потока или нет
           
            ret, frame = cap.read()
            
            try:
                # детектируем расположение лица на кадре, вероятности на сколько это лицо
                # и особенные точки лица
                boxes, probs, landmarks = self.mtcnn.detect(frame, landmarks=True)
                # Ищем лицо и рисуем рамку
                self._draw(frame, boxes, probs, landmarks)
                
                # Если в кадре есть лицо, то считываем жест:
                if boxes is not None:
                     
                    # детектируем расположение рук на кадре
                    width, height, inference_time, results = self.hand_detector.inference(frame)

                    boxes_hand = []
                    for detection in results:
                        id, name, confidence, x, y, w, h = detection
                        boxes_hand = [[x, y, x + w, y + h]]


                    # Вырезаем руку из кадра с некоторым захватом области exp=30
                    hand = self.crop_hand(frame, boxes_hand, exp=30)[0]
                    # Меняем размер изображения руки для входа в нейронную сеть
                    hand = cv2.resize(hand,(100,100))
                    # Превращаем в 1-канальное серое изображение
                    hand = cv2.cvtColor(hand, cv2.COLOR_BGR2GRAY)
                    # Превращаем numpy-картинку вырезанной руки в pytorch-тензор
                    torch_hand = torch.from_numpy(hand).unsqueeze(0).to(self.device).float()

                    # Загужаем наш тензор руки в нейронную сеть и получаем предсказание
                    gesture = self.emodel(torch_hand[None, ...])               
                    # Заполняем таблици жестов
                    guess_list[int(gesture.argmax())] += 1

            except:
                pass
                #print('Something wrong im main cycle!')
            
            # Получаем жест с наибольшим значением 
            if max(guess_list) >= 4:
                gesture_idx = guess_list.argmax()
                guess_list = np.array([0] * 10)
            
            # Реакция на наш жест (вывод названия жеста на экран и изменение параметров выводимого кадра)
            gesture_to_show, changed_frame = self.gesture_action(gesture_idx, frame)
            cv2.putText(changed_frame, gesture_to_show, (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (188, 198, 48), 2, cv2.LINE_AA)
            # Показываем кадр в окне, и назвываем его(окно) - 'Gesture Recognition'
            cv2.imshow('Gesture Recognition', changed_frame)
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            if (cv2.waitKey(1) & 0xFF == ord('q')):
                break

        # Очищаем все объекты opencv, что мы создали
        cap.release()
        cv2.destroyAllWindows()
        
        
# Загружаем мтцнн
mtcnn = MTCNN()
# Создаем объект нашего класса приложения
fcd = FaceDetector(mtcnn)
# Запускаем
fcd.run()

C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
